# Engineering time series features using tsfresh

In [24]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import tsfresh
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from functools import reduce

# format notebook output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML, Javascript
display(HTML("<style>.container { width:90% !important; }</style>"))

# style pandas display
pd.set_option('display.max_columns', None)

# matplotlib magic
%matplotlib inline

In [3]:
def desc(df):
    display(df.head())
    print(df.shape)
    print(df.dtypes)

# Load data

In [4]:
air_reserve = pd.read_csv('../../data_files/air_reserve.csv')
air_store_info = pd.read_csv('../../data_files/air_store_info.csv', encoding='utf-8')
air_visit_data = pd.read_csv('../../data_files/air_visit_data.csv')
date_info = pd.read_csv('../../data_files/date_info.csv')
hpg_reserve = pd.read_csv('../../data_files/hpg_reserve.csv')
hpg_store_info = pd.read_csv('../../data_files/hpg_store_info.csv')
sample_submission = pd.read_csv('../../data_files/sample_submission.csv')
store_id_relation = pd.read_csv('../../data_files/store_id_relation.csv')

### Will's data

In [5]:
xover_visit_res = pd.read_csv('/Users/eric.bunch/Downloads/df_air_hpg_visit_reserve_821.csv', encoding='utf-8')

In [6]:
xover_visit_res['visit_date'] = pd.to_datetime(xover_visit_res['visit_date'])

In [7]:
xover_visit_res['reserve_date'] = pd.to_datetime(xover_visit_res['reserve_date'])

In [8]:
desc(xover_visit_res)

,air_store_id,visit_date,reserve_date,visitors,reserve_visitors_total
0,air_fff68b929994bfbd,2017-04-22,2017-04-22,5,0
1,air_fff68b929994bfbd,2017-04-22,2017-04-21,5,0
2,air_fff68b929994bfbd,2017-04-22,2017-04-20,5,0
3,air_fff68b929994bfbd,2017-04-22,2017-04-19,5,0
4,air_fff68b929994bfbd,2017-04-22,2017-04-18,5,0


(39636238, 5)
air_store_id                      object
visit_date                datetime64[ns]
reserve_date              datetime64[ns]
visitors                           int64
reserve_visitors_total             int64
dtype: object


## Merge data

In [9]:
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].apply(lambda x: x[:10])
# air_reserve['reserve_date'] = pd.to_datetime(air_reserve['reserve_date'])

In [10]:
air_reserve['visit_date'] = air_reserve['visit_datetime'].apply(lambda x: x[:10])
# air_reserve['visit_date'] = pd.to_datetime(air_reserve['visit_date'])

In [11]:
air_reserve_daily = air_reserve[[
        'air_store_id',
        'visit_date',
        'reserve_visitors'
    ]].groupby(['air_store_id', 'visit_date'], as_index=False).agg('sum')

In [12]:
desc(air_reserve_daily)

,air_store_id,visit_date,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-05,9
2,air_00a91d42b08b08d9,2016-12-14,18
3,air_00a91d42b08b08d9,2016-12-17,2
4,air_00a91d42b08b08d9,2016-12-20,4


(29830, 3)
air_store_id        object
visit_date          object
reserve_visitors     int64
dtype: object


In [13]:
air_visit_data = pd.merge(
    left=air_visit_data,
    right=air_reserve_daily,
    on=['air_store_id', 'visit_date'],
    how='left',
    copy=False
)

In [14]:
# should we make a seperate column with binary for nullfill?
air_visit_data['reserve_visitors'].fillna(0, inplace=True)

In [15]:
air_visit_data['visit_date'] = pd.to_datetime(air_visit_data['visit_date'])

In [16]:
desc(air_visit_data)

,air_store_id,visit_date,visitors,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,0.0
1,air_ba937bf13d40fb24,2016-01-14,32,0.0
2,air_ba937bf13d40fb24,2016-01-15,29,0.0
3,air_ba937bf13d40fb24,2016-01-16,22,0.0
4,air_ba937bf13d40fb24,2016-01-18,6,0.0


(252108, 4)
air_store_id                object
visit_date          datetime64[ns]
visitors                     int64
reserve_visitors           float64
dtype: object


# Create rolling time series dataframes

In [17]:
def drop_features(df):
    df.dropna(axis=1, inplace=True)
    
    zero_variance_features = df.var().iloc[np.where(df.var() == 0)].index
    return df[[col for col in df.columns if col not in zero_variance_features]]

In [18]:
def fill_missing_dates(df):
    """
    helper function to fill in dates that are missing from the data set
    in order to have a contiguous segment of dates for each location id
    """
    
    min_date = df['visit_date'].min()
    max_date = df['visit_date'].max()
    air_store_id = df['air_store_id'].values[0]
    
    idx = pd.date_range(min_date, max_date)
    
    return df.set_index('visit_date').reindex(idx).reset_index().rename(columns={
        'index': 'visit_date'
    }).fillna(value={
        'air_store_id': air_store_id,
        'visitors': 0
    })

In [19]:
def clean_ext_df(df, location_id, wndw):
    df = df.rename(columns={
        col: col+'_wndw_'+str(wndw) for col in df.columns
    }).reset_index().rename(columns={'id': 'visit_date'})
    
    df.columns.name = None
    
    df['air_store_id'] = location_id
    
    return df

In [20]:
air_store_ids = air_visit_data['air_store_id'].unique()

In [21]:
air_store_dict = {
    air_store_id: air_visit_data[
        air_visit_data['air_store_id'] == air_store_id
    ][[
        'air_store_id',
        'visit_date',
        'visitors'
    ]] 
    for air_store_id in air_store_ids[0:2]
}

In [22]:
air_store_dict = {
    air_store_id: fill_missing_dates(air_store_dict[air_store_id])
    for air_store_id in air_store_dict.keys()
}

## 7 day window

In [25]:
%%time

# The functionality to call roll_time_series() on a dictionary is not working properly.
# The max_timeshift parameter has no effect on the output in that case (it should); it 
# always executes as if max_timeshift=None.

tsroll_7 = {
    air_store_id: tsfresh.utilities.dataframe_functions.roll_time_series(
        df_or_dict=air_store_dict[air_store_id],
        column_id='air_store_id',
        column_sort='visit_date',
        column_kind=None,
        rolling_direction=1,
        max_timeshift=6 # rolling window of 1 week
    )
    for air_store_id in air_store_dict.keys()
}

CPU times: user 42.7 ms, sys: 2.16 ms, total: 44.8 ms
Wall time: 43.1 ms


In [26]:
for key in tsroll_7.keys():
    print(tsroll_7[key].shape)

(3241, 3)
(3297, 3)


In [27]:
%%time

tsroll_7 = {
    air_store_id: pd.merge(
        left=tsroll_7[air_store_id],
        right=xover_visit_res[
            xover_visit_res['air_store_id'] == air_store_id
        ][[
            'visit_date',
            'reserve_date',
            'reserve_visitors_total'
        ]].rename(columns={
            'visit_date': 'air_store_id',
            'reserve_date': 'visit_date'
        }),
        on=['visit_date', 'air_store_id'],
        how='left',
        copy=False
    )
    for air_store_id in tsroll_7.keys()
}

CPU times: user 6.24 s, sys: 567 ms, total: 6.81 s
Wall time: 6.8 s


In [28]:
# shape check to make sure merge did not introduce duplicates
for key in tsroll_7.keys():
    print(tsroll_7[key].shape)

(3241, 4)
(3297, 4)


In [29]:
# Example
## This is windowed data for the location with id air_ba937bf13d40fb24 on the date 2016-05-31.
## That is, it contains the data in a window of 7 dates back (including the date 2016-05-31)
## for the location air_ba937bf13d40fb24

## IPORTANT ##
## This window contains data for the specified date, i.e. for 2016-05-31. 
## So we would use this information to train the model to predict on 
## the date 2016-06-01.

tsroll_7['air_ba937bf13d40fb24'][
    tsroll_7['air_ba937bf13d40fb24']['air_store_id'] == '2016-05-31'
]

tsroll_7['air_ba937bf13d40fb24'][
    tsroll_7['air_ba937bf13d40fb24']['air_store_id'] == '2016-05-31'
].shape

,visit_date,visitors,air_store_id,reserve_visitors_total
952,2016-05-25,33.0,2016-05-31,0
953,2016-05-26,26.0,2016-05-31,0
954,2016-05-27,30.0,2016-05-31,0
955,2016-05-28,27.0,2016-05-31,0
956,2016-05-29,0.0,2016-05-31,0
957,2016-05-30,17.0,2016-05-31,0
958,2016-05-31,25.0,2016-05-31,0


(7, 4)

In [30]:
%%time

# The option to pass in a dictionary of dataframes is not working properly. 
# After passing dictionary with two dataframes, and calling dropna(axis=1) in the 
# resulting extracted features, there are no columns left. There should be at least 
# as many columns as there were left after calling dropna(axis=1) in the 
# extracted features from just passing in a single dataframe (which was a non-zero number).

ext_7 = {
    air_store_id: extract_features(
        tsroll_7[air_store_id],
        column_id="air_store_id",
        column_sort="visit_date"
    )
    for air_store_id in tsroll_7.keys()
}

Feature Extraction: 100%|██████████| 20/20 [00:17<00:00,  1.16it/s]


CPU times: user 7.58 s, sys: 744 ms, total: 8.32 s
Wall time: 38.6 s


In [31]:
ext_7 = {
    air_store_id: drop_features(ext_7[air_store_id])
    for air_store_id in ext_7.keys()
}

In [32]:
for key in ext_7.keys():
    print(key, ext_7[key].shape)

air_ba937bf13d40fb24 (466, 166)
air_25e9888d30b386df (474, 168)


In [33]:
ext_7 = {
    air_store_id: clean_ext_df(df=ext_7[air_store_id], location_id=air_store_id, wndw=7)
    for air_store_id in ext_7.keys()
}

In [34]:
e7 = pd.concat(
    objs=list(ext_7.values()),
    ignore_index=True
)

In [35]:
e7.head()
e7.shape

,air_store_id,reserve_visitors_total__has_duplicate_max_wndw_7,reserve_visitors_total__has_duplicate_min_wndw_7,reserve_visitors_total__has_duplicate_wndw_7,reserve_visitors_total__length_wndw_7,reserve_visitors_total__longest_strike_above_mean_wndw_7,reserve_visitors_total__longest_strike_below_mean_wndw_7,reserve_visitors_total__percentage_of_reoccurring_datapoints_to_all_datapoints_wndw_7,reserve_visitors_total__range_count__max_1__min_-1_wndw_7,reserve_visitors_total__ratio_value_number_to_time_series_length_wndw_7,reserve_visitors_total__value_count__value_0_wndw_7,visit_date,visitors__abs_energy_wndw_7,visitors__absolute_sum_of_changes_wndw_7,"visitors__agg_autocorrelation__f_agg_""mean""_wndw_7","visitors__agg_autocorrelation__f_agg_""median""_wndw_7","visitors__agg_autocorrelation__f_agg_""var""_wndw_7",visitors__approximate_entropy__m_2__r_0.1_wndw_7,visitors__approximate_entropy__m_2__r_0.3_wndw_7,visitors__approximate_entropy__m_2__r_0.5_wndw_7,visitors__approximate_entropy__m_2__r_0.7_wndw_7,visitors__approximate_entropy__m_2__r_0.9_wndw_7,visitors__binned_entropy__max_bins_10_wndw_7,visitors__c3__lag_1_wndw_7,visitors__c3__lag_2_wndw_7,visitors__c3__lag_3_wndw_7,"visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.2__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.6_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2_w

(940, 170)

## 30 day window

In [38]:
%%time

# The functionality to call roll_time_series() on a dictionary is not working properly.
# The max_timeshift parameter has no effect on the output in that case (it should); it 
# always executes as if max_timeshift=None.

tsroll_30 = {
    air_store_id: tsfresh.utilities.dataframe_functions.roll_time_series(
        df_or_dict=air_store_dict[air_store_id],
        column_id='air_store_id',
        column_sort='visit_date',
        column_kind=None,
        rolling_direction=1,
        max_timeshift=30 # rolling window of 1 month
    )
    for air_store_id in air_store_dict.keys()
}

CPU times: user 143 ms, sys: 5.67 ms, total: 148 ms
Wall time: 147 ms


In [39]:
%%time

tsroll_30 = {
    air_store_id: pd.merge(
        left=tsroll_30[air_store_id],
        right=xover_visit_res[
            xover_visit_res['air_store_id'] == air_store_id
        ][[
            'visit_date',
            'reserve_date',
            'reserve_visitors_total'
        ]].rename(columns={
            'visit_date': 'air_store_id',
            'reserve_date': 'visit_date'
        }),
        on=['visit_date', 'air_store_id'],
        how='left',
        copy=False
    )
    for air_store_id in tsroll_30.keys()
}

CPU times: user 5.47 s, sys: 34.3 ms, total: 5.5 s
Wall time: 5.49 s


In [40]:
%%time

# The option to pass in a dictionary of dataframes is not working properly. 
# After passing dictionary with two dataframes, and calling dropna(axis=1) in the 
# resulting extracted features, there are no columns left. There should be at least 
# as many columns as there were left after calling dropna(axis=1) in the 
# extracted features from just passing in a single dataframe (which was a non-zero number).

ext_30 = {
    air_store_id: extract_features(
        tsroll_30[air_store_id],
        column_id="air_store_id",
        column_sort="visit_date"
    )
    for air_store_id in tsroll_30.keys()
}

Feature Extraction: 100%|██████████| 20/20 [00:21<00:00,  1.10s/it]


CPU times: user 7.39 s, sys: 770 ms, total: 8.16 s
Wall time: 47.9 s


In [41]:
ext_30 = {
    air_store_id: drop_features(ext_30[air_store_id])
    for air_store_id in ext_30.keys()
}

In [42]:
for key in ext_30.keys():
    print(key, ext_30[key].shape)

air_ba937bf13d40fb24 (466, 192)
air_25e9888d30b386df (474, 192)


In [43]:
ext_30 = {
    air_store_id: clean_ext_df(df=ext_30[air_store_id], location_id=air_store_id, wndw=30)
    for air_store_id in ext_30.keys()
}

In [44]:
e30 = pd.concat(
    objs=list(ext_30.values()),
    ignore_index=True
)

In [45]:
e30.head()
e30.shape

,visit_date,reserve_visitors_total__has_duplicate_wndw_30,reserve_visitors_total__has_duplicate_max_wndw_30,reserve_visitors_total__has_duplicate_min_wndw_30,reserve_visitors_total__length_wndw_30,reserve_visitors_total__longest_strike_above_mean_wndw_30,reserve_visitors_total__longest_strike_below_mean_wndw_30,reserve_visitors_total__percentage_of_reoccurring_datapoints_to_all_datapoints_wndw_30,reserve_visitors_total__range_count__max_1__min_-1_wndw_30,reserve_visitors_total__ratio_value_number_to_time_series_length_wndw_30,reserve_visitors_total__value_count__value_0_wndw_30,visitors__abs_energy_wndw_30,visitors__absolute_sum_of_changes_wndw_30,"visitors__agg_autocorrelation__f_agg_""mean""_wndw_30","visitors__agg_autocorrelation__f_agg_""median""_wndw_30","visitors__agg_autocorrelation__f_agg_""var""_wndw_30",visitors__approximate_entropy__m_2__r_0.1_wndw_30,visitors__approximate_entropy__m_2__r_0.3_wndw_30,visitors__approximate_entropy__m_2__r_0.5_wndw_30,visitors__approximate_entropy__m_2__r_0.7_wndw_30,visitors__approximate_entropy__m_2__r_0.9_wndw_30,visitors__binned_entropy__max_bins_10_wndw_30,visitors__c3__lag_1_wndw_30,visitors__c3__lag_2_wndw_30,visitors__c3__lag_3_wndw_30,"visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.2__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.4_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.4_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.6_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.4_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6_wndw_30","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8_wndw_30","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0_wndw_30","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.2_wndw_30","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0_wndw_30","visitors__change_quantile

(940, 194)

## 365 day window

In [48]:
%%time

# The functionality to call roll_time_series() on a dictionary is not working properly.
# The max_timeshift parameter has no effect on the output in that case (it should); it 
# always executes as if max_timeshift=None.

tsroll_365 = {
    air_store_id: tsfresh.utilities.dataframe_functions.roll_time_series(
        df_or_dict=air_store_dict[air_store_id],
        column_id='air_store_id',
        column_sort='visit_date',
        column_kind=None,
        rolling_direction=1,
        max_timeshift=364 # rolling window of 1 year
    )
    for air_store_id in air_store_dict.keys()
}

CPU times: user 1.44 s, sys: 11.2 ms, total: 1.46 s
Wall time: 1.46 s


In [49]:
%%time

tsroll_365 = {
    air_store_id: pd.merge(
        left=tsroll_365[air_store_id],
        right=xover_visit_res[
            xover_visit_res['air_store_id'] == air_store_id
        ][[
            'visit_date',
            'reserve_date',
            'reserve_visitors_total'
        ]].rename(columns={
            'visit_date': 'air_store_id',
            'reserve_date': 'visit_date'
        }),
        on=['visit_date', 'air_store_id'],
        how='left',
        copy=False
    ).fillna(0)
    for air_store_id in tsroll_365.keys()
}

CPU times: user 5.63 s, sys: 65.9 ms, total: 5.7 s
Wall time: 5.68 s


In [50]:
%%time

# The option to pass in a dictionary of dataframes is not working properly. 
# After passing dictionary with two dataframes, and calling dropna(axis=1) in the 
# resulting extracted features, there are no columns left. There should be at least 
# as many columns as there were left after calling dropna(axis=1) in the 
# extracted features from just passing in a single dataframe (which was a non-zero number).

ext_365 = {
    air_store_id: extract_features(
        tsroll_365[air_store_id],
        column_id="air_store_id",
        column_sort="visit_date"
    )
    for air_store_id in tsroll_365.keys()
}

Feature Extraction: 100%|██████████| 20/20 [01:02<00:00,  3.11s/it]


CPU times: user 8.03 s, sys: 813 ms, total: 8.84 s
Wall time: 2min 6s


In [51]:
ext_365 = {
    air_store_id: drop_features(ext_365[air_store_id])
    for air_store_id in ext_365.keys()
}

In [52]:
for key in ext_365.keys():
    print(key, ext_365[key].shape)

air_ba937bf13d40fb24 (466, 192)
air_25e9888d30b386df (474, 196)


In [53]:
ext_365 = {
    air_store_id: clean_ext_df(df=ext_365[air_store_id], location_id=air_store_id, wndw=365)
    for air_store_id in ext_365.keys()
}

In [54]:
e365 = pd.concat(
    objs=list(ext_365.values()),
    ignore_index=True
)

In [55]:
e365.head()
e365.shape

,air_store_id,reserve_visitors_total__has_duplicate_max_wndw_365,reserve_visitors_total__has_duplicate_min_wndw_365,reserve_visitors_total__has_duplicate_wndw_365,reserve_visitors_total__length_wndw_365,reserve_visitors_total__longest_strike_above_mean_wndw_365,reserve_visitors_total__longest_strike_below_mean_wndw_365,reserve_visitors_total__percentage_of_reoccurring_datapoints_to_all_datapoints_wndw_365,reserve_visitors_total__range_count__max_1__min_-1_wndw_365,reserve_visitors_total__ratio_value_number_to_time_series_length_wndw_365,reserve_visitors_total__value_count__value_0_wndw_365,visit_date,visitors__abs_energy_wndw_365,visitors__absolute_sum_of_changes_wndw_365,"visitors__agg_autocorrelation__f_agg_""mean""_wndw_365","visitors__agg_autocorrelation__f_agg_""median""_wndw_365","visitors__agg_autocorrelation__f_agg_""var""_wndw_365",visitors__approximate_entropy__m_2__r_0.1_wndw_365,visitors__approximate_entropy__m_2__r_0.3_wndw_365,visitors__approximate_entropy__m_2__r_0.5_wndw_365,visitors__approximate_entropy__m_2__r_0.7_wndw_365,visitors__approximate_entropy__m_2__r_0.9_wndw_365,visitors__binned_entropy__max_bins_10_wndw_365,visitors__c3__lag_1_wndw_365,visitors__c3__lag_2_wndw_365,visitors__c3__lag_3_wndw_365,"visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.2__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.4_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.4_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.6_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.4_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6_wndw_365","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8_wndw_365","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0_wndw_365","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.2_wndw_365","visitors__change_quantiles__f_agg_""

(940, 198)

## Features over all time

In [56]:
%%time

# The functionality to call roll_time_series() on a dictionary is not working properly.
# The max_timeshift parameter has no effect on the output in that case (it should); it 
# always executes as if max_timeshift=None.

tsroll_alltime = {
    air_store_id: tsfresh.utilities.dataframe_functions.roll_time_series(
        df_or_dict=air_store_dict[air_store_id],
        column_id='air_store_id',
        column_sort='visit_date',
        column_kind=None,
        rolling_direction=1,
        max_timeshift=None # no constraint on rolling window
    )
    for air_store_id in air_store_dict.keys()
}

CPU times: user 1.97 s, sys: 21.8 ms, total: 1.99 s
Wall time: 2 s


In [57]:
%%time

tsroll_alltime = {
    air_store_id: pd.merge(
        left=tsroll_alltime[air_store_id],
        right=xover_visit_res[
            xover_visit_res['air_store_id'] == air_store_id
        ][[
            'visit_date',
            'reserve_date',
            'reserve_visitors_total'
        ]].rename(columns={
            'visit_date': 'air_store_id',
            'reserve_date': 'visit_date'
        }),
        on=['visit_date', 'air_store_id'],
        how='left',
        copy=False
    ).fillna(0)
    for air_store_id in tsroll_alltime.keys()
}

CPU times: user 5.57 s, sys: 62.6 ms, total: 5.63 s
Wall time: 5.62 s


In [58]:
%%time

# The option to pass in a dictionary of dataframes is not working properly. 
# After passing dictionary with two dataframes, and calling dropna(axis=1) in the 
# resulting extracted features, there are no columns left. There should be at least 
# as many columns as there were left after calling dropna(axis=1) in the 
# extracted features from just passing in a single dataframe (which was a non-zero number).

ext_alltime = {
    air_store_id: extract_features(
        tsroll_alltime[air_store_id],
        column_id="air_store_id",
        column_sort="visit_date"
    )
    for air_store_id in tsroll_alltime.keys()
}

Feature Extraction: 100%|██████████| 20/20 [01:09<00:00,  3.50s/it]


CPU times: user 7.66 s, sys: 826 ms, total: 8.49 s
Wall time: 2min 17s


In [59]:
ext_alltime = {
    air_store_id: drop_features(ext_alltime[air_store_id])
    for air_store_id in ext_alltime.keys()
}

In [60]:
for key in ext_alltime.keys():
    print(key, ext_alltime[key].shape)

air_ba937bf13d40fb24 (466, 192)
air_25e9888d30b386df (474, 194)


In [61]:
ext_alltime = {
    air_store_id: clean_ext_df(df=ext_alltime[air_store_id], location_id=air_store_id, wndw='alltime')
    for air_store_id in ext_alltime.keys()
}

In [62]:
ealltime = pd.concat(
    objs=list(ext_alltime.values()),
    ignore_index=True
)

In [63]:
ealltime.head()
ealltime.shape

,air_store_id,reserve_visitors_total__has_duplicate_max_wndw_alltime,reserve_visitors_total__has_duplicate_min_wndw_alltime,reserve_visitors_total__has_duplicate_wndw_alltime,reserve_visitors_total__length_wndw_alltime,reserve_visitors_total__longest_strike_above_mean_wndw_alltime,reserve_visitors_total__longest_strike_below_mean_wndw_alltime,reserve_visitors_total__percentage_of_reoccurring_datapoints_to_all_datapoints_wndw_alltime,reserve_visitors_total__range_count__max_1__min_-1_wndw_alltime,reserve_visitors_total__ratio_value_number_to_time_series_length_wndw_alltime,reserve_visitors_total__value_count__value_0_wndw_alltime,visit_date,visitors__abs_energy_wndw_alltime,visitors__absolute_sum_of_changes_wndw_alltime,"visitors__agg_autocorrelation__f_agg_""mean""_wndw_alltime","visitors__agg_autocorrelation__f_agg_""median""_wndw_alltime","visitors__agg_autocorrelation__f_agg_""var""_wndw_alltime",visitors__approximate_entropy__m_2__r_0.1_wndw_alltime,visitors__approximate_entropy__m_2__r_0.3_wndw_alltime,visitors__approximate_entropy__m_2__r_0.5_wndw_alltime,visitors__approximate_entropy__m_2__r_0.7_wndw_alltime,visitors__approximate_entropy__m_2__r_0.9_wndw_alltime,visitors__binned_entropy__max_bins_10_wndw_alltime,visitors__c3__lag_1_wndw_alltime,visitors__c3__lag_2_wndw_alltime,visitors__c3__lag_3_wndw_alltime,"visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.2__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.2_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.2_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.4_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.4_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.6_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.4_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6_wndw_alltime","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8_wndw_alltime","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql

(940, 196)

# Merge all windowed feature sets together

In [64]:
e7.shape
e30.shape
e365.shape
ealltime.shape

(940, 170)

(940, 194)

(940, 198)

(940, 196)

In [65]:
dfs = [
    e7,
    e30,
    e365,
    ealltime
]

In [66]:
df_final = reduce(lambda left,right: pd.merge(left=left, right=right, on=['air_store_id', 'visit_date'], how='left'), dfs)

In [68]:
df_final = df_final[[
    'air_store_id',
    'visit_date'
] + [
    col for col in df_final.columns if col not in ['air_store_id','visit_date']
]]

In [69]:
df_final.shape
df_final.head(2)

(940, 752)

,air_store_id,visit_date,reserve_visitors_total__has_duplicate_max_wndw_7,reserve_visitors_total__has_duplicate_min_wndw_7,reserve_visitors_total__has_duplicate_wndw_7,reserve_visitors_total__length_wndw_7,reserve_visitors_total__longest_strike_above_mean_wndw_7,reserve_visitors_total__longest_strike_below_mean_wndw_7,reserve_visitors_total__percentage_of_reoccurring_datapoints_to_all_datapoints_wndw_7,reserve_visitors_total__range_count__max_1__min_-1_wndw_7,reserve_visitors_total__ratio_value_number_to_time_series_length_wndw_7,reserve_visitors_total__value_count__value_0_wndw_7,visitors__abs_energy_wndw_7,visitors__absolute_sum_of_changes_wndw_7,"visitors__agg_autocorrelation__f_agg_""mean""_wndw_7","visitors__agg_autocorrelation__f_agg_""median""_wndw_7","visitors__agg_autocorrelation__f_agg_""var""_wndw_7",visitors__approximate_entropy__m_2__r_0.1_wndw_7,visitors__approximate_entropy__m_2__r_0.3_wndw_7,visitors__approximate_entropy__m_2__r_0.5_wndw_7,visitors__approximate_entropy__m_2__r_0.7_wndw_7,visitors__approximate_entropy__m_2__r_0.9_wndw_7,visitors__binned_entropy__max_bins_10_wndw_7,visitors__c3__lag_1_wndw_7,visitors__c3__lag_2_wndw_7,visitors__c3__lag_3_wndw_7,"visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.2__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.6_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6_wndw_7","visitors__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.4_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.0_wndw_7","visitors__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2_w

In [71]:
df_final.to_csv('/Users/eric.bunch/kaggle_features_sample.csv', index=False)